In [1]:
!pip install tsfresh

  Using cached numpy-1.20.3-cp39-cp39-win_amd64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.20.3 which is incompatible.
jax 0.4.8 requires numpy>=1.21, but you have numpy 1.20.3 which is incompatible.


In [5]:
!pip install --upgrade numpy

In [2]:
import os
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, extract_features

In [14]:
# Read the DataFrame from the HDF5 file
df = pd.read_hdf(r"C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Test.h5", key='test_dataframe')

In [27]:
# Read the DataFrame from the HDF5 file
df = pd.read_hdf(r"C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Dataset.h5", key='dataset')

In [28]:
print(df)

                  Source  Time (ms)          Type  Value
0          i030520235006      0.000  Nset (1/min)  0.000
1          i030520235068      0.000  Nset (1/min)  0.000
2          i050520238018      0.000   Angle (deg)  0.000
3          i030520237070      0.000   Angle (deg)  0.000
4          i280420232085      0.000   Angle (deg)  0.000
...                  ...        ...           ...    ...
33375222  t1005202314051   5001.519    TCP_y (mm)  2.600
33375223  t1005202314051   5001.519    TCP_x (mm)  0.000
33375224  t1005202314051   5001.519  TCP_rz (rad) -1.204
33375225  t1005202314051   5001.519    TCP_z (mm) -4.600
33375226  t1005202314051   5001.519  TCP_rx (rad)  1.234

[33375227 rows x 4 columns]


In [ ]:
# Rename columns to meet tsfresh requirements
df = df.rename(columns={'Source': 'id', 'Time (ms)': 'time', 'Type': 'kind', 'Value': 'value'})
df['id'] = df['id'].str[1:]
# Define feature extraction settings
settings = MinimalFCParameters()  # or EfficientFCParameters()

# Extract features
extracted_features = extract_features(df, column_id="id", column_kind="kind", column_sort="time", column_value="value", default_fc_parameters=MinimalFCParameters())

In [20]:
print(extracted_features)

              TCP_ry (rad)__sum_values  TCP_ry (rad)__median  \
030520234000                  1014.619                 1.163   
030520234001                   930.909                 1.168   
030520235006                  1029.665                 1.166   
030520235007                   874.619                 1.171   

              TCP_ry (rad)__mean  TCP_ry (rad)__length  \
030520234000            1.163554                 872.0   
030520234001            1.162184                 801.0   
030520235006            1.166099                 883.0   
030520235007            1.166159                 750.0   

              TCP_ry (rad)__standard_deviation  TCP_ry (rad)__variance  \
030520234000                          0.008197                0.000067   
030520234001                          0.010118                0.000102   
030520235006                          0.006297                0.000040   
030520235007                          0.005659                0.000032   

              TCP

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assume that labels are in a separate dataframe
labels = pd.read_csv(r"C:\Users\GHB\Desktop\SCREW PROJECT\Machine Learning\Data\Labels_names.csv")
labels = labels.rename(columns={'File Name': 'id'})

# Create a LabelEncoder instance
encoder = LabelEncoder()

# Fit the encoder and transform the labels
labels['Label'] = encoder.fit_transform(labels['Label'])

# Print original labels and their encoded values
for original, encoded in zip(encoder.classes_, range(len(encoder.classes_))):
    print(f'Original: {original}, Encoded: {encoded}')

# Merge features and labels
data = extracted_features.merge(labels, left_index=True, right_on='id')

print(data)




Original: N, Encoded: 0
Original: NS, Encoded: 1
Original: OT, Encoded: 2
Original: P, Encoded: 3
Original: UT, Encoded: 4


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [11]:
# Split data into features and target variable
X = data.drop(columns=['Label'])
X = data.drop(columns=['id'])
y = data['Label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').